In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama
import os

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

dir = "pdf/"
metadatas = []
chunks = []  
chunk_size = 432
chunk_overlap = 8

In [3]:
for arq in os.listdir(dir):
    
    start_index = 0
    text_join = ""
    
    metadatas.append({"name": arq})
    print(arq)
    text = PdfReader(dir+arq)
    text_join = ''.join(page.extract_text() for page in text.pages)

    for i in range(0, len(text_join), chunk_size):
        start = i
        end = i + chunk_size
        if start != 0:
            start = start - chunk_overlap
            end =  end + chunk_overlap
        chunks.append(text_join[start:end])
        metadatas.append({"name": arq, "page": i})

embeddings_db = embeddings.encode(chunks)

Livro de Arquitetura e organização de computadores.pdf
Learning LangChain-O'Reilly Media (2024).pdf
JavaBasico.pdf


In [4]:
for i, d in enumerate(chunks):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadatas[i]
    )

In [ ]:
while True:

    query = []

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=10,
        )

    metadata = results['metadatas'][0]
    data = results['documents'][0]
    distance = results['distances'] 
          
    for k in range(10):
        if distance[0][k] < 0.74:
            query.append(data[k])

    prompt_template = f"""You are a chatbot specialized in generating quizzes. 
                        Generate a quiz with 5 questions in Portuguese in the following context: {query} with the following format: question, options and explanation of the right answer"""

    if len(query) == 0:
            prompt_template = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    
    output = ollama.generate(
            model="gemma",
            prompt=prompt_template
            )
    print("\n")
    output = output['response']
    print(output)
    print("\n")
    

Pergunte algo endereçamento




**1. Qual o significado de endereçamento imediato?**

A) Um endereçamento onde o valor é incluído na própria instrução.
B) Um endereçamento onde o valor está localizado em um registrador.
C) Um endereçamento onde o valor está localizado em memória.
D) Um endereçamento onde o valor é calculado em tempo de execução.

**Resposta correta:** A) Um endereçamento onde o valor é incluído na própria instrução.


**2. Qual o tipo de endereçamento que usa o valor da própria instrução?**

A) Endereçamento imediato
B) Endereçamento direto
C) Endereçamento indireto
D) Endereçamento por registrador

**Resposta correta:** A) Endereçamento imediato


**3. Qual o significado de endereçamento relativo?**

A) Um endereçamento onde o valor é calculado em relação a um endereço específico.
B) Um endereçamento onde o valor é calculado em relação ao endereço da própria instrução.
C) Um endereçamento onde o valor é calculado em relação ao endereço da memória principal.
D) Um endereçamento onde o valor é calcu

Pergunte algo objeto em orientação a objeto




**1. Qual o objetivo do encapsulamento na Orientação a Objeto?**

A) Expor todos os atributos e métodos de uma classe.
B) Ocultar atributos privados e métodos públicos.
C) Reduzir o tamanho da classe.
D) Aumentar o número de métodos públicos.

**Resposta correta:** B) Ocultar atributos privados e métodos públicos.


**2. Qual o objetivo de usar métodos públicos em uma classe?**

A) Limitar o acesso a métodos.
B) Reduzir o tamanho da classe.
C) Facilitar o acesso a métodos de outras classes.
D) Ocultar métodos de uma classe.

**Resposta correta:** C) Facilitar o acesso a métodos de outras classes.


**3. Qual o propósito do método get em uma classe?**

A) Definir o valor de um atributo privado.
B) Recuperar o valor de um atributo privado.
C) Definir o valor de um método público.
D) Recuperar o valor de um método público.

**Resposta correta:** B) Recuperar o valor de um atributo privado.


**4. Qual o propósito do método set em uma classe?**

A) Definir o valor de um atributo privado.

Pergunte algo incremento e decremento


...

Pergunte algo tipos primitivos




Desculpe, não tenho conhecimento sobre essa pergunta específica.




Pergunte algo tipos primitivos em java




**1. Qual o propósito principal do Java?**

A) Criar sistemas operacionais
B) Desenhar interfaces gráficas de usuário
C) Desenhar aplicações web
D) Desenvolvimento de aplicações móveis

**Resposta correta:** C) Desenhar aplicações web

**Explicação:** O Java foi desenvolvido para criar aplicações de computador de propósito geral, especialmente aplicações web.


**2. Qual o tipo de dados do Java usado para armazenar parâmetros?**

A) Integer
B) String
C) Float
D) Date

**Resposta correta:** B) String

**Explicação:** O tipo de dados String do Java é usado para armazenar dados de texto.


**3. Qual é o conjunto de bibliotecas básicas do Java?**

A) Java Standard Edition
B) Java Collections Framework
C) Java AWT
D) Java Swing

**Resposta correta:** A) Java Standard Edition

**Explicação:** O Java Standard Edition contém as bibliotecas básicas do Java.


**4. Qual é o nome da primeira plataforma do Java?**

A) Core
B) AWT
C) Swing
D) Java Native Access

**Resposta correta:** A) Core

**E

Pergunte algo estruturas de dados em java
